In [2]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
#!pip install emoji
import emoji as emoji
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import tensorflow_hub as hub




In [7]:
train_X = pd.read_csv('../input/customdataset/emojify_train_x.csv',header=None)
test_X = pd.read_csv('../input/customdataset/emojiy_test_x.csv',header=None)

train_Y = pd.read_csv('../input/customdataset/Emojify_Y_train.csv',header=None)
test_Y = pd.read_csv('../input/customdataset/emojiy_y_test.csv',header=None)


In [8]:
frames = [train_X , test_X]
X = pd.concat(frames)
X.shape

In [9]:
frames = [train_Y , test_Y]
Y = pd.concat(frames)
Y.shape

In [10]:
import re as s
def clean(train):
  List = train.iloc[:,0].tolist()
  Regex = str.maketrans("","","'")
  word = [s.translate(Regex) for s in List]
  tokenized_sent = []
  for s in word:
    tokenized_sent.append(word_tokenize(s.lower())) 
  return word , tokenized_sent


trained_list , trained_and_tokened_list = clean(X)

In [11]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model_e = hub.load(module_url)
print ("module %s loaded" % module_url)

sentence_embeddings = model_e(trained_list)


In [12]:
category = Y.iloc[:,0].tolist()
category_ohe = pd.get_dummies(category)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings.numpy(), category_ohe, test_size=0.33, random_state=42)

In [14]:
from keras.layers import *
from keras.models import Sequential
model = Sequential()
model.add(Dense(64, input_dim=512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['acc'])
model.fit(X_train,y_train,epochs=40,shuffle=True,validation_split=0.1)

In [15]:
model.evaluate(X_test,y_test)

In [16]:
import emoji as emoji
#emoji.EMOJI_ALIAS_UNICODE

emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }


In [17]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

In [18]:
query = ['I am gping to temple']
inp = model_e(query)
out =  model.predict(inp).flatten().tolist()

max_value = max(out)
max_index = out.index(max_value)

print(out)
print(max_value)
print(max_index)



print('Statement is :  '+query[0])
print('Classification Result is :  '+emoji.emojize(emoji_dictionary[str(max_index)]))

